## Lookup Table for British Airways

In [14]:
import pandas as pd

# Step 1: Load Excel file
file_path = "British Airways Summer Schedule Dataset - Forage Data Science Task 1.xlsx"
df = pd.read_excel(file_path, sheet_name="british_airways_schedule_summer")

# Step 2: Clean column names
df.columns = df.columns.str.strip().str.upper()

# Step 3: Check if required columns are present
required_columns = [
    "ARRIVAL_REGION", "HAUL", "TIME_OF_DAY",
    "FIRST_CLASS_SEATS", "BUSINESS_CLASS_SEATS", "ECONOMY_SEATS",
    "TIER1_ELIGIBLE_PAX", "TIER2_ELIGIBLE_PAX", "TIER3_ELIGIBLE_PAX"
]

if not all(col in df.columns for col in required_columns):
    raise ValueError("Required columns are missing in the dataset.")

# Step 4: Calculate actual total passengers per flight
df["TOTAL_PAX"] = (
    df["FIRST_CLASS_SEATS"] +
    df["BUSINESS_CLASS_SEATS"] +
    df["ECONOMY_SEATS"]
)

# Step 5: Group by REGION + HAUL + TIME
grouped = df.groupby(["ARRIVAL_REGION", "HAUL", "TIME_OF_DAY"]).agg({
    "TIER1_ELIGIBLE_PAX": "mean",
    "TIER2_ELIGIBLE_PAX": "mean",
    "TIER3_ELIGIBLE_PAX": "mean",
    "TOTAL_PAX": "mean"
}).reset_index()

# Step 6: Calculate % of total passengers eligible per lounge tier
grouped["Tier 1 % (Concorde)"] = (grouped["TIER1_ELIGIBLE_PAX"] / grouped["TOTAL_PAX"] * 100).round(1)
grouped["Tier 2 % (First Lounge)"] = (grouped["TIER2_ELIGIBLE_PAX"] / grouped["TOTAL_PAX"] * 100).round(1)
grouped["Tier 3 % (Club Lounge)"] = (grouped["TIER3_ELIGIBLE_PAX"] / grouped["TOTAL_PAX"] * 100).round(1)

# Step 7: Total Lounge Eligible %
grouped["Total Lounge Eligible %"] = (
    grouped["Tier 1 % (Concorde)"] +
    grouped["Tier 2 % (First Lounge)"] +
    grouped["Tier 3 % (Club Lounge)"]
).round(1)

# Step 8: Select final output columns
lookup_table = grouped[[
    "ARRIVAL_REGION", "HAUL", "TIME_OF_DAY",
    "Tier 1 % (Concorde)", "Tier 2 % (First Lounge)", "Tier 3 % (Club Lounge)", "Total Lounge Eligible %"
]].sort_values(by="Total Lounge Eligible %", ascending=False).reset_index(drop=True)

# Step 9: Save to Excel
lookup_table.to_excel("BA_Lounge_Eligibility_Lookup.xlsx", index=False)

# Step 10: Show result
print(lookup_table.head())

  ARRIVAL_REGION   HAUL TIME_OF_DAY  Tier 1 % (Concorde)  \
0         Europe  SHORT   Lunchtime                  0.4   
1         Europe  SHORT     Evening                  0.3   
2         Europe  SHORT     Morning                  0.3   
3         Europe  SHORT   Afternoon                  0.3   
4           Asia   LONG   Afternoon                  0.2   

   Tier 2 % (First Lounge)  Tier 3 % (Club Lounge)  Total Lounge Eligible %  
0                      4.5                    17.3                     22.2  
1                      4.5                    17.0                     21.8  
2                      4.4                    16.7                     21.4  
3                      4.3                    16.6                     21.2  
4                      2.9                    10.8                     13.9  
